In [43]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics import mean_squared_error
plt.rcParams['font.family'] = 'Apple SD Gothic Neo'
plt.rcParams['axes.unicode_minus'] = False
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정
# pd.read_csv() 함수를 사용해서 데이터를 읽어오는 코드입니다.
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# pd.read_csv() 함수를 사용해서 데이터를 읽어오는 코드입니다.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [11]:
import numpy as np
from sklearn.metrics import (
    log_loss,
    accuracy_score,
    classification_report,
    mean_squared_error,
    r2_score,
)

def evaluate_model(model, X_valid, y_valid, name="model", verbose=True):
    y_pred = model.predict(X_valid)

    # Log Loss (로스 로스, log_loss)
    loss = None
    if hasattr(model, "predict_proba"):
        try:
            y_proba = model.predict_proba(X_test)
            loss = log_loss(X_valid, y_proba)
        except Exception:
            loss = None

    # R² (결정계수, R-squared)
    r2 = r2_score(y_valid, y_pred)

    # 정확도(Accuracy)
    acc = None
    cls_report = None
    try:
        acc = accuracy_score(y_valid, y_pred)
        cls_report = classification_report(y_valid, y_pred)
    except Exception:
        # 회귀모델(continuous target)일 때는 여기로 옴
        pass

    # MSE / RMSE
    mse = mean_squared_error(y_valid, y_pred)
    rmse = np.sqrt(mse)

    # ------------ 출력 (verbose=True일 때만) ------------
    if verbose:
        print("=" * 40)
        print(f"[{name}] 모델 성능 평가")
        print("=" * 40)

        if loss is not None:
            print("Log Loss:", loss)

        print(f"{name:15s} R score(): {r2:.4f}")

        if acc is not None:
            print("정확도:", acc)
            if cls_report is not None:
                print(cls_report)

        print("-" * 40)
        print(f"MSE (평균 제곱 오차): {mse:.3f}")
        print(f"RMSE (평균 제곱근 오차): {rmse:.3f}")
        print(f"R² Score (결정계수): {r2:.3f}")
        print("=" * 40)

    # 핵심: 정확도(없으면 None)를 리턴
    return acc

In [12]:
train.rename(columns={'혈압': '맥압'}, inplace=True)
test.rename(columns={'혈압': '맥압'}, inplace=True)

In [13]:
train.loc[train['시력'] > 2, '시력'] = 2.0
train.loc[train['혈청 크레아티닌'] > 1.5, '혈청 크레아티닌'] = 1.5
train.loc[train['요 단백'] > 4, '요 단백'] = 4.0
train.loc[train['저밀도지단백'] > 150, '저밀도지단백'] = 150
train.loc[train['고밀도지단백'] > 100, '고밀도지단백'] = 100
train.loc[train['중성 지방'] > 200, '중성 지방'] = 200
train.loc[train['공복 혈당'] > 200, '공복 혈당'] = 200

train.loc[train['저밀도지단백'] < 22, '저밀도지단백'] = 22
train.loc[train['고밀도지단백'] < 22, '고밀도지단백'] = 22


In [15]:
test.loc[test['시력'] > 2, '시력'] = 2.0
test.loc[test['혈청 크레아티닌'] > 1.5, '혈청 크레아티닌'] = 1.5
test.loc[test['요 단백'] > 4, '요 단백'] = 4.0
test.loc[test['저밀도지단백'] > 150, '저밀도지단백'] = 150
test.loc[test['고밀도지단백'] > 100, '고밀도지단백'] = 100
test.loc[test['중성 지방'] > 200, '중성 지방'] = 200
test.loc[test['공복 혈당'] > 200, '공복 혈당'] = 200
test.loc[test['간 효소율'] > 10, '간 효소율'] = 10

test.loc[test['저밀도지단백'] < 22, '저밀도지단백'] = 22
test.loc[test['고밀도지단백'] < 22, '고밀도지단백'] = 22


In [17]:
train['저밀도지단백_고밀도지단백_낮음'] = train['저밀도지단백'] / train['고밀도지단백']
train['고밀도지단백_저밀도지단백_높음'] = train['고밀도지단백'] / train['저밀도지단백']
train['중성지방_고밀도지단백_2이하'] = train['중성 지방'] / train['고밀도지단백']
train['true_BMI'] = train['몸무게(kg)'] / ((train['키(cm)'] / 100) ** 2)
train['총 콜레스테롤'] = train['저밀도지단백'] +train['고밀도지단백'] + (train['중성 지방'] / 5)



In [18]:
test['저밀도지단백_고밀도지단백_낮음'] = test['저밀도지단백'] / test['고밀도지단백']
test['고밀도지단백_저밀도지단백_높음'] = test['고밀도지단백'] / test['저밀도지단백']
test['중성지방_고밀도지단백_2이하'] = test['중성 지방'] / test['고밀도지단백']
test['true_BMI'] = test['몸무게(kg)'] / ((test['키(cm)'] / 100) ** 2)
test['총 콜레스테롤'] = test['저밀도지단백'] +test['고밀도지단백'] + (test['중성 지방'] / 5)


In [19]:
train.drop(columns=['BMI','콜레스테롤'], inplace=True)

In [20]:
test.drop(columns=['BMI','콜레스테롤'], inplace=True)

In [21]:
X = train.drop(['label','ID',], axis = 1)
y = train['label']
X_test = test.drop(['ID',], axis = 1)

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


---

# 모델링

In [39]:
# 선형 모델 (Linear Models)
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀

# 거리 기반 (Distance-based)
from sklearn.neighbors import KNeighborsClassifier   # k-최근접 이웃

# 트리 기반 (Tree-based)
from sklearn.tree import DecisionTreeClassifier      # 결정나무
from sklearn.ensemble import RandomForestClassifier  # 랜덤 포레스트
from sklearn.ensemble import GradientBoostingClassifier  # 그래디언트 부스팅

# 서포트 벡터 머신 (SVM)
from sklearn.svm import SVC                          # 서포트 벡터 분류
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.base import clone
plt.rcParams['font.family'] = 'Apple SD Gothic Neo'
plt.rcParams['axes.unicode_minus'] = False


In [40]:
models = [
    ('LogisticRegression',LogisticRegression(max_iter=1000, random_state=42)),
    ('DecisionTreeClassifier',DecisionTreeClassifier(random_state=42)),
    ('RandomForestClassifier',RandomForestClassifier(n_estimators=100, random_state=42)),
    ('GradientBoostingClassifier',GradientBoostingClassifier(random_state=42)),
    ('SVC',SVC(probability=True, random_state=42)),
    ('GradientBoostingClassifier',GradientBoostingClassifier(random_state=42)),
    ('XGBClassifier',XGBClassifier(random_state=42)),
]

results = []

for name,base_model in models:
    model = clone(base_model)
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_valid_pred = model.predict(X_valid)

    train_acc = accuracy_score(y_train, y_train_pred)
    valid_acc = accuracy_score(y_valid, y_valid_pred)

    cv_scores = cross_val_score(
        base_model, X, y, cv=5, scoring='accuracy'
        )
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    results.append({
        "name": name,
        "train_acc": train_acc,
        "valid_acc": valid_acc,
        "cv_mean": cv_mean,
        "cv_std": cv_std,
    })

    results_sorted = sorted(results, key=lambda x: x["valid_acc"], reverse=True)

for r in results_sorted:
    gap = r["train_acc"] - r["valid_acc"]
    print(
                f"{r['name']}\n"
        f"  Train Acc: {r['train_acc']}\n"
        f"  Valid Acc: {r['valid_acc']}\n"
        f"  Gap(Train-Valid): {gap}\n"
        f"  CV Mean: {r['cv_mean']} (+/- {r['cv_std']})\n"
    )


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

RandomForestClassifier
  Train Acc: 1.0
  Valid Acc: 0.745
  Gap(Train-Valid): 0.255
  CV Mean: 0.7312857142857143 (+/- 0.008273389848897288)

GradientBoostingClassifier
  Train Acc: 0.77375
  Valid Acc: 0.7364285714285714
  Gap(Train-Valid): 0.037321428571428616
  CV Mean: 0.7282857142857143 (+/- 0.005707138387268055)

GradientBoostingClassifier
  Train Acc: 0.77375
  Valid Acc: 0.7364285714285714
  Gap(Train-Valid): 0.037321428571428616
  CV Mean: 0.7282857142857143 (+/- 0.005707138387268055)

XGBClassifier
  Train Acc: 0.9864285714285714
  Valid Acc: 0.7192857142857143
  Gap(Train-Valid): 0.2671428571428571
  CV Mean: 0.726 (+/- 0.005795846507171663)

SVC
  Train Acc: 0.6992857142857143
  Valid Acc: 0.7157142857142857
  Gap(Train-Valid): -0.01642857142857146
  CV Mean: 0.7009999999999998 (+/- 0.014530755559944491)

LogisticRegression
  Train Acc: 0.7067857142857142
  Valid Acc: 0.715
  Gap(Train-Valid): -0.00821428571428573
  CV Mean: 0.7064285714285714 (+/- 0.00769309258162075)

De

---

# 1차 성능 평가

```python
RandomForestClassifier
  Train Acc: 1.0
  Valid Acc: 0.745
  Gap(Train-Valid): 0.255
  CV Mean: 0.7312857142857143 (+/- 0.008273389848897288)

GradientBoostingClassifier
  Train Acc: 0.77375
  Valid Acc: 0.7364285714285714
  Gap(Train-Valid): 0.037321428571428616
  CV Mean: 0.7282857142857143 (+/- 0.005707138387268055)

GradientBoostingClassifier
  Train Acc: 0.77375
  Valid Acc: 0.7364285714285714
  Gap(Train-Valid): 0.037321428571428616
  CV Mean: 0.7282857142857143 (+/- 0.005707138387268055)

XGBClassifier
  Train Acc: 0.9864285714285714
  Valid Acc: 0.7192857142857143
  Gap(Train-Valid): 0.2671428571428571
  CV Mean: 0.726 (+/- 0.005795846507171663)

SVC
  Train Acc: 0.6992857142857143
  Valid Acc: 0.7157142857142857
  Gap(Train-Valid): -0.01642857142857146
  CV Mean: 0.7009999999999998 (+/- 0.014530755559944491)

LogisticRegression
  Train Acc: 0.7067857142857142
  Valid Acc: 0.715
  Gap(Train-Valid): -0.00821428571428573
  CV Mean: 0.7064285714285714 (+/- 0.00769309258162075)

DecisionTreeClassifier
  Train Acc: 1.0
  Valid Acc: 0.665
  Gap(Train-Valid): 0.33499999999999996
  CV Mean: 0.6777142857142857 (+/- 0.007946247991527669)

```

---

# 2차 성능 평가 전처리x

In [44]:
X = train.drop(['label','ID',], axis = 1)
y = train['label']
X_test = test.drop(['ID',], axis = 1)

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [45]:
models = [
    ('LogisticRegression',LogisticRegression(max_iter=1000, random_state=42)),
    ('DecisionTreeClassifier',DecisionTreeClassifier(random_state=42)),
    ('RandomForestClassifier',RandomForestClassifier(n_estimators=100, random_state=42)),
    ('GradientBoostingClassifier',GradientBoostingClassifier(random_state=42)),
    ('SVC',SVC(probability=True, random_state=42)),
    ('GradientBoostingClassifier',GradientBoostingClassifier(random_state=42)),
    ('XGBClassifier',XGBClassifier(random_state=42)),
]

results = []

for name,base_model in models:
    model = clone(base_model)
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_valid_pred = model.predict(X_valid)

    train_acc = accuracy_score(y_train, y_train_pred)
    valid_acc = accuracy_score(y_valid, y_valid_pred)

    cv_scores = cross_val_score(
        base_model, X, y, cv=5, scoring='accuracy'
        )
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    results.append({
        "name": name,
        "train_acc": train_acc,
        "valid_acc": valid_acc,
        "cv_mean": cv_mean,
        "cv_std": cv_std,
    })

    results_sorted = sorted(results, key=lambda x: x["valid_acc"], reverse=True)

for r in results_sorted:
    gap = r["train_acc"] - r["valid_acc"]
    print(
                f"{r['name']}\n"
        f"  Train Acc: {r['train_acc']}\n"
        f"  Valid Acc: {r['valid_acc']}\n"
        f"  Gap(Train-Valid): {gap}\n"
        f"  CV Mean: {r['cv_mean']} (+/- {r['cv_std']})\n"
    )


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

RandomForestClassifier
  Train Acc: 1.0
  Valid Acc: 0.735
  Gap(Train-Valid): 0.265
  CV Mean: 0.7390000000000001 (+/- 0.005952190473142742)

LogisticRegression
  Train Acc: 0.7046428571428571
  Valid Acc: 0.7235714285714285
  Gap(Train-Valid): -0.018928571428571406
  CV Mean: 0.707 (+/- 0.008752842104023082)

GradientBoostingClassifier
  Train Acc: 0.7728571428571429
  Valid Acc: 0.7235714285714285
  Gap(Train-Valid): 0.04928571428571438
  CV Mean: 0.7241428571428572 (+/- 0.007047940790596602)

GradientBoostingClassifier
  Train Acc: 0.7728571428571429
  Valid Acc: 0.7235714285714285
  Gap(Train-Valid): 0.04928571428571438
  CV Mean: 0.7241428571428572 (+/- 0.007047940790596602)

XGBClassifier
  Train Acc: 0.9825
  Valid Acc: 0.7228571428571429
  Gap(Train-Valid): 0.2596428571428572
  CV Mean: 0.7172857142857143 (+/- 0.003539860483818271)

SVC
  Train Acc: 0.695
  Valid Acc: 0.7121428571428572
  Gap(Train-Valid): -0.017142857142857237
  CV Mean: 0.6962857142857143 (+/- 0.011878638003

```python
RandomForestClassifier
  Train Acc: 1.0
  Valid Acc: 0.735
  Gap(Train-Valid): 0.265
  CV Mean: 0.7390000000000001 (+/- 0.005952190473142742)

LogisticRegression
  Train Acc: 0.7046428571428571
  Valid Acc: 0.7235714285714285
  Gap(Train-Valid): -0.018928571428571406
  CV Mean: 0.707 (+/- 0.008752842104023082)

GradientBoostingClassifier
  Train Acc: 0.7728571428571429
  Valid Acc: 0.7235714285714285
  Gap(Train-Valid): 0.04928571428571438
  CV Mean: 0.7241428571428572 (+/- 0.007047940790596602)

GradientBoostingClassifier
  Train Acc: 0.7728571428571429
  Valid Acc: 0.7235714285714285
  Gap(Train-Valid): 0.04928571428571438
  CV Mean: 0.7241428571428572 (+/- 0.007047940790596602)

XGBClassifier
  Train Acc: 0.9825
  Valid Acc: 0.7228571428571429
  Gap(Train-Valid): 0.2596428571428572
  CV Mean: 0.7172857142857143 (+/- 0.003539860483818271)

SVC
  Train Acc: 0.695
  Valid Acc: 0.7121428571428572
  Gap(Train-Valid): -0.017142857142857237
  CV Mean: 0.6962857142857143 (+/- 0.011878638003421404)

DecisionTreeClassifier
  Train Acc: 1.0
  Valid Acc: 0.6764285714285714
  Gap(Train-Valid): 0.3235714285714286
  CV Mean: 0.672 (+/- 0.010855262995150225)
```

# 비교 표

| 모델                         | 1차 Train Acc(훈련 정확도) | 1차 Valid Acc(검증 정확도) | 1차 Gap(Train-Valid) | 1차 CV Mean | 2차 Train Acc | 2차 Valid Acc | 2차 Gap(Train-Valid) | 2차 CV Mean |
| -------------------------- | -------------------- | -------------------- | ------------------- | ---------- | ------------ | ------------ | ------------------- | ---------- |
| RandomForestClassifier     | 1.0000               | 0.7450               | 0.2550              | 0.7313     | 1.0000       | 0.7350       | 0.2650              | 0.7390     |
| GradientBoostingClassifier | 0.7738               | 0.7364               | 0.0373              | 0.7283     | 0.7729       | 0.7236       | 0.0493              | 0.7241     |
| XGBClassifier              | 0.9864               | 0.7193               | 0.2671              | 0.7260     | 0.9825       | 0.7229       | 0.2596              | 0.7173     |
| SVC                        | 0.6993               | 0.7157               | -0.0164             | 0.7010     | 0.6950       | 0.7121       | -0.0171             | 0.6963     |
| LogisticRegression         | 0.7068               | 0.7150               | -0.0082             | 0.7064     | 0.7046       | 0.7236       | -0.0189             | 0.7070     |
| DecisionTreeClassifier     | 1.0000               | 0.6650               | 0.3350              | 0.6777     | 1.0000       | 0.6764       | 0.3236              | 0.6720     |


| 모델                           | 1차 Valid | 2차 Valid | 차이(1차−2차)   | 해석                 |
| ---------------------------- | -------- | -------- | ----------- | ------------------ |
| RandomForest (랜덤 포레스트)       | 0.7450   | 0.7350   | **+0.0100** | 파생변수가 약간 도움        |
| GradientBoosting (그래디언트 부스팅) | 0.7364   | 0.7236   | **+0.0129** | 파생변수 덕분에 ~1.3%p 상승 |
| XGB                          | 0.7193   | 0.7229   | **−0.0036** | 거의 비슷, 2차가 약간 우위   |
| SVC                          | 0.7157   | 0.7121   | **+0.0036** | 미세하게 1차가 낫지만 거의 동일 |
| LogisticRegression (로지스틱 회귀) | 0.7150   | 0.7236   | **−0.0086** | 원시데이터 쪽이 더 잘 맞음    |
| DecisionTree (단일 트리)         | 0.6650   | 0.6764   | **−0.0114** | 원시데이터 쪽이 조금 더 나음   |


---
# 예측

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

probs = models.predict_proba(X_valid)[:, 1]

best_acc = 0
best_thr = 0.5

for thr in np.linspace(0.1, 0.9, 17):  # 0.1 ~ 0.9 사이 0.05 간격
    y_pred_thr = (probs >= thr).astype(int)
    acc = accuracy_score(y_valid, y_pred_thr)
    if acc > best_acc:
        best_acc = acc
        best_thr = thr

print("Best threshold:", best_thr)
print("Best val accuracy:", best_acc)